In [5]:
from ultralytics import YOLO
import cv2
import numpy as np

In [ ]:
def detect_green_blob(image):
    """
    Detects green blobs in the given image.

    Args:
        image (np.array): The BGR image (crop of the detected object).

    Returns:
        (bool, tuple): A tuple where the first element is True if a green blob is found,
                       and the second element is the bounding box (x, y, w, h) of the blob
                       relative to the cropped image.
    """
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])

    # Create a mask that captures areas with green color
    mask = cv2.inRange(hsv, lower_green, upper_green)

   
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Loop over contours and check if any are large enough to be considered a blob
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 30:  # Only consider blobs with area greater than 100 pixels
            x, y, w, h = cv2.boundingRect(cnt)
            return True, (x, y, w, h)
    return False, None


## Working code

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

def detect_green_blob(image):
    """
    Detects green blobs in the given image.

    Args:
        image (np.array): The BGR image (crop of the detected object).

    Returns:
        (bool, tuple): A tuple where the first element is True if a green blob is found,
                       and the second element is the bounding box (x, y, w, h) of the blob
                       relative to the cropped image.
    """
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

   
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 30:
            x, y, w, h = cv2.boundingRect(cnt)
            return True, (x, y, w, h)
    return False, None

# Load the YOLO model
model = YOLO('weights/best.pt')

source = 'scene_5.mkv'
cap = cv2.VideoCapture(source)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_video_path = "results/scene_5_yolov11s.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process the video frame-by-frame
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    results = model(frame, conf=0.85)
    annotated_frame = results[0].plot()

    if results[0].boxes is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame_width, x2)
            y2 = min(frame_height, y2)
            cropped_obj = frame[y1:y2, x1:x2]
            if cropped_obj.size == 0:
                continue

            # Detect if there is any green blob in the cropped object
            green_found, green_bbox = detect_green_blob(cropped_obj)
            if green_found:
                # green_bbox is relative to the cropped object; adjust to frame coordinates.
                gx, gy, gw, gh = green_bbox
                cv2.rectangle(
                    annotated_frame,
                    (x1 + gx, y1 + gy),
                    (x1 + gx + gw, y1 + gy + gh),
                    (0, 255, 0),
                    2
                )
                cv2.putText(
                    annotated_frame,
                    "Green Blob",
                    (x1 + gx, y1 + gy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2
                )

    out.write(annotated_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 237.2ms
Speed: 3.0ms preprocess, 237.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 244.0ms
Speed: 2.0ms preprocess, 244.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 323.4ms
Speed: 39.2ms preprocess, 323.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 199.4ms
Speed: 2.0ms preprocess, 199.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 233.6ms
Speed: 3.0ms preprocess, 233.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 183.8ms
Speed: 3.4ms preprocess, 183.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 191.5ms
Speed: 3.9ms preprocess, 191.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 195.0ms
Speed: 2.5ms prep

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

def detect_green_blob(image):
    """
    Detects green blobs in the given image.

    Args:
        image (np.array): The BGR image (crop of the detected object).

    Returns:
        (bool, list): A tuple where the first element is True if at least one green blob is found,
                      and the second element is a list of bounding boxes (x, y, w, h) of each blob
                      relative to the cropped image.
    """
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 40, 40])
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Apply morphological operations to remove noise
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours of the green areas
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    green_bboxes = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 30:
            x, y, w, h = cv2.boundingRect(cnt)
            green_bboxes.append((x, y, w, h))
    return (True, green_bboxes) if green_bboxes else (False, [])

# Load the YOLO model
model = YOLO('weights/best.pt')

source = 'scene_5.mkv'
cap = cv2.VideoCapture(source)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_video_path = "results/scene_5_updated_blob_detection_yolov11s.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Process the video frame-by-frame
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Run YOLO detection on the frame
    results = model(frame, conf=0.85)
    annotated_frame = results[0].plot()

    if results[0].boxes is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            # Ensure the bounding box is within frame boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame_width, x2)
            y2 = min(frame_height, y2)
            cropped_obj = frame[y1:y2, x1:x2]
            if cropped_obj.size == 0:
                continue

            # Detect all green blobs within the cropped object
            green_found, green_bboxes = detect_green_blob(cropped_obj)
            if green_found:
                for green_bbox in green_bboxes:
                    gx, gy, gw, gh = green_bbox
                    # Adjust the coordinates relative to the full frame and draw the bounding box
                    cv2.rectangle(
                        annotated_frame,
                        (x1 + gx, y1 + gy),
                        (x1 + gx + gw, y1 + gy + gh),
                        (0, 255, 0),
                        2
                    )
                    cv2.putText(
                        annotated_frame,
                        "Green Blob",
                        (x1 + gx, y1 + gy - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0, 255, 0),
                        2
                    )

    out.write(annotated_frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 455.3ms
Speed: 8.2ms preprocess, 455.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 385.7ms
Speed: 12.5ms preprocess, 385.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 340.3ms
Speed: 3.0ms preprocess, 340.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 306.9ms
Speed: 4.0ms preprocess, 306.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 524.4ms
Speed: 3.0ms preprocess, 524.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 462.1ms
Speed: 38.6ms preprocess, 462.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 316.3ms
Speed: 6.2ms preprocess, 316.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 312.6ms
Speed: 4.0ms pre